In [4]:
import pandas as pd

In [5]:
# pip install openpyxl 

Note: you may need to restart the kernel to use updated packages.


# Using the full table

In [6]:
dt = pd.read_excel("Total Table.xlsx")
dt

,customer_id,order_date,product_name,price,member
0,A,2021-01-01,sushi,10,N
1,A,2021-01-01,curry,15,N
2,A,2021-01-07,curry,15,Y
3,A,2021-01-10,ramen,12,Y
4,A,2021-01-11,ramen,12,Y
5,A,2021-01-11,ramen,12,Y
6,B,2021-01-01,curry,15,N
7,B,2021-01-02,curry,15,N
8,B,2021-01-04,sushi,10,N
9,B,2021-01-11,sushi,10,Y


# 1. What is the total amount each customer spent at the restaurant?

In [7]:
dt.groupby("customer_id").sum().reset_index().rename(columns={"customer_id":"Customer ID","price":"Amount Spent"})


,Customer ID,Amount Spent
0,A,76
1,B,74
2,C,36


# 2. How many days has each customer visited the restaurant?

In [8]:
dt[["customer_id","price"]].groupby("customer_id").count().reset_index(drop=False).rename(columns={"customer_id":"Customer ID","price":"Number of Visits"})

,Customer ID,Number of Visits
0,A,6
1,B,6
2,C,3


# 3. What was the first item from the menu purchased by each customer?

In [9]:
d3 = dt[["customer_id","order_date"]].groupby(["customer_id"]).min()
d3.reset_index(inplace=True)
# len(d3.values)
print(d3)
print(d3.values)
d3

  customer_id order_date
0           A 2021-01-01
1           B 2021-01-01
2           C 2021-01-01
[['A' Timestamp('2021-01-01 00:00:00')]
 ['B' Timestamp('2021-01-01 00:00:00')]
 ['C' Timestamp('2021-01-01 00:00:00')]]


,customer_id,order_date
0,A,2021-01-01
1,B,2021-01-01
2,C,2021-01-01


In [10]:
# dt[dt["customer_id"] == "A" ].groupby(["product_name"]).min()
dt[dt["customer_id"] == "A" ].agg(("min","max"))[["product_name","order_date"]]

,product_name,order_date
min,curry,2021-01-01
max,sushi,2021-01-11


In [11]:
for i in range(len(d3.values)):
    print(dt[(dt["customer_id"] == d3.values[i][0]) & (dt["order_date"] == d3.values[i][1])])


  customer_id order_date product_name  price member
0           A 2021-01-01        sushi     10      N
1           A 2021-01-01        curry     15      N
  customer_id order_date product_name  price member
6           B 2021-01-01        curry     15      N
   customer_id order_date product_name  price member
12           C 2021-01-01        ramen     12      N
13           C 2021-01-01        ramen     12      N


# 4. What is the most purchased item on the menu and how many times was it purchased by all customers?

In [12]:
dt.groupby("product_name").count().reset_index()[["product_name","customer_id"]].rename(columns={"product_name":"Product Name","customer_id":"Count"})

,Product Name,Count
0,curry,4
1,ramen,8
2,sushi,3


# 5. Which item was the most popular for each customer?

In [13]:
d4 = dt.pivot_table(index="product_name", columns="customer_id", values="price", aggfunc="count")
d4

customer_id,A,B,C
product_name,,,
curry,2.0,2.0,NaN
ramen,3.0,2.0,3.0
sushi,1.0,2.0,NaN


In [14]:
d4 = dt.groupby(["customer_id","product_name"]).count().reset_index()[["customer_id","product_name","order_date"]].rename(columns={"customer_id":"Customer ID","product_name":"Product Name","order_date":"Count"})
# dt[].agg(("count"))[["customer_id","product_name"]]
d4.sort_values(by=["Customer ID","Count"], ascending=[True,False])

,Customer ID,Product Name,Count
1,A,ramen,3
0,A,curry,2
2,A,sushi,1
3,B,curry,2
4,B,ramen,2
5,B,sushi,2
6,C,ramen,3


# 6. Which item was purchased first by the customer after they became a member?

In [15]:
dx = pd.read_csv("members.csv")
dt = dt.merge(dx,how="left", on="customer_id")

In [16]:
print(dt)
print(dx)
print(dx.loc[0])
print(dx.loc[0][1])
print(dx.loc[0,"join_date"])

   customer_id order_date product_name  price member   join_date
0            A 2021-01-01        sushi     10      N  2021-01-07
1            A 2021-01-01        curry     15      N  2021-01-07
2            A 2021-01-07        curry     15      Y  2021-01-07
3            A 2021-01-10        ramen     12      Y  2021-01-07
4            A 2021-01-11        ramen     12      Y  2021-01-07
5            A 2021-01-11        ramen     12      Y  2021-01-07
6            B 2021-01-01        curry     15      N  2021-01-09
7            B 2021-01-02        curry     15      N  2021-01-09
8            B 2021-01-04        sushi     10      N  2021-01-09
9            B 2021-01-11        sushi     10      Y  2021-01-09
10           B 2021-01-16        ramen     12      Y  2021-01-09
11           B 2021-02-01        ramen     12      Y  2021-01-09
12           C 2021-01-01        ramen     12      N         NaN
13           C 2021-01-01        ramen     12      N         NaN
14           C 2021-01-07

In [17]:
# dt[(dt["customer_id"]==i) & (dt["order_date"]>=dt["join_date"])]
# dt[(dt["customer_id"]=='A') & (dt["member"] == "Y")]
dt_i = dt[(dt["customer_id"]=='A') & (dt["member"] == "Y")]
# dt_i[dt_i["order_date"] == dt_i["order_date"].agg("min")]
dt_i[dt_i["order_date"] == dt_i["order_date"].min()]

,customer_id,order_date,product_name,price,member,join_date
2,A,2021-01-07,curry,15,Y,2021-01-07


In [18]:
# dx.customer_id
# dt["order_date"].min()
for i in dx["customer_id"]:
    print(i)

A
B


In [19]:
# for i in dx.customer_id:
for i in dx["customer_id"]:
    dt_i = dt[(dt["customer_id"]== i) & (dt["member"] == "Y")]
    print(dt_i[dt_i["order_date"] == dt_i["order_date"].agg("min")])
# dt

  customer_id order_date product_name  price member   join_date
2           A 2021-01-07        curry     15      Y  2021-01-07
  customer_id order_date product_name  price member   join_date
9           B 2021-01-11        sushi     10      Y  2021-01-09


# 7. Which item was purchased just before the customer became a member?

In [20]:
# dt[(dt["customer_id"] == "A") & (dt["order_date"] < dt["join_date"])]
dt_j = dt[(dt["customer_id"] == "A") & (dt["member"] == "N")]
dt_j[dt_j["order_date"] == dt_j["order_date"].agg("max")]
# dt_j[dt_j["order_date"] == dt_j["order_date"].max()]
# dt

,customer_id,order_date,product_name,price,member,join_date
0,A,2021-01-01,sushi,10,N,2021-01-07
1,A,2021-01-01,curry,15,N,2021-01-07


In [21]:
for i in dx.customer_id:
    dt_j = dt[(dt["customer_id"] == i) & (dt["member"] == "N")]
    print(dt_j[dt_j["order_date"] == dt_j["order_date"].agg("max")])



  customer_id order_date product_name  price member   join_date
0           A 2021-01-01        sushi     10      N  2021-01-07
1           A 2021-01-01        curry     15      N  2021-01-07
  customer_id order_date product_name  price member   join_date
8           B 2021-01-04        sushi     10      N  2021-01-09


# 8. What is the total items and amount spent for each member before they became a member?

In [22]:
d8 = dt[(dt["member"] == "N") & ((dt["customer_id"] == "A")|(dt["customer_id"] == "B"))][["customer_id","product_name","price"]]
d8.groupby(["customer_id","product_name"]).agg(("sum","count")).reset_index()


customer_id product_name price      
                             sum count
0           A        curry    15     1
1           A        sushi    10     1
2           B        curry    30     2
3           B        sushi    10     1

In [23]:
dt[(dt["member"] == "N") & ((dt["customer_id"] == "A")|(dt["customer_id"] == "B"))][["customer_id","product_name","price"]].groupby(["customer_id","product_name"]).agg(("sum","count")).reset_index()
# dt[(dt["member"] == "N") & ((dt["customer_id"] == "A")|(dt["customer_id"] == "B"))][["customer_id","product_name","price"]]

customer_id product_name price      
                             sum count
0           A        curry    15     1
1           A        sushi    10     1
2           B        curry    30     2
3           B        sushi    10     1

# 9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?

In [24]:
dt["multiplier"] = None
for i in range(len(dt)):
    if dt["product_name"][i] == "sushi":
        dt["multiplier"][i] = 2
    else:
        dt["multiplier"][i] = 1
dt["points"] = dt["price"] * dt["multiplier"] * 10

dt

C:\Users\aduno\AppData\Local\Temp\ipykernel_26456\4288908663.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt["multiplier"][i] = 2
C:\Users\aduno\AppData\Local\Temp\ipykernel_26456\4288908663.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt["multiplier"][i] = 1


,customer_id,order_date,product_name,price,member,join_date,multiplier,points
0,A,2021-01-01,sushi,10,N,2021-01-07,2,200
1,A,2021-01-01,curry,15,N,2021-01-07,1,150
2,A,2021-01-07,curry,15,Y,2021-01-07,1,150
3,A,2021-01-10,ramen,12,Y,2021-01-07,1,120
4,A,2021-01-11,ramen,12,Y,2021-01-07,1,120
5,A,2021-01-11,ramen,12,Y,2021-01-07,1,120
6,B,2021-01-01,curry,15,N,2021-01-09,1,150
7,B,2021-01-02,curry,15,N,2021-01-09,1,150
8,B,2021-01-04,sushi,10,N,2021-01-09,2,200
9,B,2021-01-11,sushi,10,Y,2021-01-09,2,200


In [25]:
d9 = dt[["customer_id","points"]].groupby(["customer_id"]).sum().reset_index()
d9

,customer_id,points
0,A,860
1,B,940
2,C,360


# 10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [26]:
#Converting to datetime object
pd.to_datetime(dx.loc[0,"join_date"])


Timestamp('2021-01-07 00:00:00')

In [27]:
# Adding time to a datetime object
import datetime
pd.to_datetime(dx.loc[0,"join_date"]) + datetime.timedelta(days=7)

Timestamp('2021-01-14 00:00:00')

In [28]:
pd.to_datetime(dt["order_date"][i]).month

1

In [29]:
import datetime
dt["multiplier2"] = None
for i in range(len(dt)):
    if (pd.to_datetime(dt["join_date"][i]) + datetime.timedelta(days=7)) >= pd.to_datetime(dt["order_date"][i]) >= pd.to_datetime(dt["join_date"][i]) and pd.to_datetime(dt["order_date"][i]).month == 1:
        dt["multiplier2"][i] = 2
    else:
        dt["multiplier2"][i] = 1

dt["points2"] = dt["price"] * dt["multiplier2"] * 10
dt
 

C:\Users\aduno\AppData\Local\Temp\ipykernel_26456\289392770.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt["multiplier2"][i] = 1
C:\Users\aduno\AppData\Local\Temp\ipykernel_26456\289392770.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt["multiplier2"][i] = 2


,customer_id,order_date,product_name,price,member,join_date,multiplier,points,multiplier2,points2
0,A,2021-01-01,sushi,10,N,2021-01-07,2,200,1,100
1,A,2021-01-01,curry,15,N,2021-01-07,1,150,1,150
2,A,2021-01-07,curry,15,Y,2021-01-07,1,150,2,300
3,A,2021-01-10,ramen,12,Y,2021-01-07,1,120,2,240
4,A,2021-01-11,ramen,12,Y,2021-01-07,1,120,2,240
5,A,2021-01-11,ramen,12,Y,2021-01-07,1,120,2,240
6,B,2021-01-01,curry,15,N,2021-01-09,1,150,1,150
7,B,2021-01-02,curry,15,N,2021-01-09,1,150,1,150
8,B,2021-01-04,sushi,10,N,2021-01-09,2,200,1,100
9,B,2021-01-11,sushi,10,Y,2021-01-09,2,200,2,200


In [30]:
d10 = dt[["customer_id","points2"]].groupby(["customer_id"]).sum().reset_index()
d10

,customer_id,points2
0,A,1270
1,B,960
2,C,360
